In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [2]:
events = pd.read_csv("events_up_to_01062018.csv", low_memory=False)
training_labels = pd.read_csv("labels_training_set.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)

In [3]:
training_labels.sort_values(by = 'person', inplace=True)
training_labels.reset_index(drop=True,inplace=True)

In [4]:
events_contados = events.groupby('person')['event'].count().to_frame().reset_index()
# events_contados = events_contados.drop(events_contados[~events_contados['person'].isin(training_labels['person'])].index) 
events_contados.sort_values(by = 'person', inplace=True)
events_contados.reset_index(drop=True,inplace=True)
events_contados.head()

,person,event
0,0008ed71,6
1,00091926,448
2,00091a7a,10
3,000ba417,206
4,000c79fe,17


In [141]:
events_data = events.pivot_table(index='person', columns='event', values='timestamp', aggfunc='count', fill_value=0)
events_data.columns = events_data.columns.astype('object')
events_data.reset_index(inplace=True)
final_data = pd.merge(events_contados, events_data, on ='person', how = 'inner')
final_data.head()

,person,event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,6,0,0,3,0,1,0,0,0,0,0,2
1,00091926,448,15,25,2,0,0,0,0,0,0,372,34
2,00091a7a,10,1,5,0,0,0,0,0,0,0,3,1
3,000ba417,206,1,24,6,1,14,0,1,0,0,153,6
4,000c79fe,17,1,0,1,0,1,0,1,9,0,3,1


In [159]:
set_entrenamiento = pd.merge(final_data, training_labels, on ='person', how = 'inner')
set_entrenamiento['label'] = training_labels['label']
set_test = pd.merge(final_data,labels_predict,on = "person", how = "right")
set_test.head()

,person,event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,00091926,448,15,25,2,0,0,0,0,0,0,372,34
1,00091a7a,10,1,5,0,0,0,0,0,0,0,3,1
2,000ba417,206,1,24,6,1,14,0,1,0,0,153,6
3,000e4d9e,411,19,17,1,0,17,0,5,0,0,339,13
4,000e619d,68,6,11,1,0,8,0,3,6,0,28,5


In [160]:
X, y = set_entrenamiento.iloc[:,:-1],set_entrenamiento.iloc[:,-1]
X.drop(columns='person', inplace=True)

In [161]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [162]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

# Prueba XGBoost

In [163]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [164]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [165]:
preds = xg_reg.predict(X_test)

In [166]:
personas = set_test["person"]
set_test = set_test.drop(columns=['person'])
predicts_kaggel = xg_reg.predict(set_test)
predicts_kaggel = predicts_kaggel.tolist()
df_kaggel = pd.DataFrame(predicts_kaggel)
df_kaggel.head()

,0
0,0.214547
1,0.243671
2,0.220114
3,0.220234
4,0.206901


In [167]:
df_kaggel = df_kaggel[0].map(lambda x: 1 if x > 0.5 else 0);
df_kaggel = df_kaggel.to_frame()
df_kaggel["person"] = personas
df_kaggel.rename(columns = {0: 'label'},inplace = True) 

In [168]:
df_kaggel = df_kaggel[['person','label']]

In [169]:
df_kaggel['label'].value_counts()

0    19415
Name: label, dtype: int64

# Prueba KNN

In [170]:
def KNN(k):
    knn = KNeighborsClassifier(n_neighbors=k,metric='euclidean')
    knn.fit(X_train, y_train)
    pred = knn.predict(X_test)
    print (accuracy_score(pred, y_test))
    preds_kaggel_knn = knn.predict(set_test)
    preds_kaggel_knn = preds_kaggel_knn.tolist()
    df_kaggel_knn = pd.DataFrame(preds_kaggel_knn)
    df_kaggel_knn["person"] = personas
    df_kaggel_knn.rename(columns = {0: 'label'},inplace = True) 
    df_kaggel_knn = df_kaggel_knn[['person','label']]
    print (df_kaggel_knn["label"].value_counts())

In [171]:
for x in range(1,15):
    print (x)
    KNN(x)

1
0.908060777749163
0    18428
1      987
Name: label, dtype: int64
2
0.9474633015709503
0    19361
1       54
Name: label, dtype: int64
3
0.9407674478496009
0    19252
1      163
Name: label, dtype: int64
4
0.9492660314190059
0    19395
1       20
Name: label, dtype: int64
5
0.9482359000772599
0    19367
1       48
Name: label, dtype: int64
6
0.950296162760752
0    19411
1        4
Name: label, dtype: int64
7
0.9500386299253155
0    19403
1       12
Name: label, dtype: int64
8
0.950811228431625
0    19415
Name: label, dtype: int64
9
0.950811228431625
0    19412
1        3
Name: label, dtype: int64
10
0.950811228431625
0    19415
Name: label, dtype: int64
11
0.950811228431625
0    19414
1        1
Name: label, dtype: int64
12
0.950811228431625
0    19415
Name: label, dtype: int64
13
0.950811228431625
0    19415
Name: label, dtype: int64
14
0.950811228431625
0    19415
Name: label, dtype: int64


In [187]:
knn = KNeighborsClassifier(n_neighbors=2,metric='euclidean')
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
print (accuracy_score(pred, y_test))
pred

0.9474633015709503


array([0, 0, 0, ..., 0, 0, 0])

In [188]:
accuracy_score(pred, y_test)

0.9474633015709503

In [189]:
preds_kaggel_knn = knn.predict(set_test)

In [190]:
preds_kaggel_knn = preds_kaggel_knn.tolist()
df_kaggel_knn = pd.DataFrame(preds_kaggel_knn)
df_kaggel_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19415 entries, 0 to 19414
Data columns (total 1 columns):
0    19415 non-null int64
dtypes: int64(1)
memory usage: 151.8 KB


In [191]:
# df_kaggel_knn = df_kaggel_knn.to_frame()
df_kaggel_knn["person"] = personas
df_kaggel_knn.rename(columns = {0: 'label'},inplace = True) 
df_kaggel_knn = df_kaggel_knn[['person','label']]

In [192]:
df_kaggel_knn["label"].value_counts()

0    19361
1       54
Name: label, dtype: int64

In [193]:
df_kaggel.to_csv('Resultados2.csv',index=False)